In [41]:
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input
from keras.layers.embeddings import Embedding

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.initializers import RandomNormal

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
! unzip /content/drive/MyDrive/All/Participants_Data_DCW.zip

Archive:  /content/drive/MyDrive/All/Participants_Data_DCW.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [43]:
df = pd.read_csv('/content/train.csv')

In [44]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [45]:
def clean_text(text):
    text = text.lower()
    
    
    text = re.sub(r'@[A-Za-z0-9_]+','', text)
    
    # Dealing with URL links
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex,'urlplaceholder', text)
    # A lot of url are write as follows: http bit.ly. Apply Regex for these cases
    utl_regex_2 = 'http [a-zA-Z]+\.[a-zA-Z]+'
    text = re.sub(utl_regex_2,'urlplaceholder', text)
    # Other formats: http : //t.co/ihW64e8Z
    utl_regex_3 = 'http \: //[a-zA-Z]\.(co|com|pt|ly)/[A-Za-z0-9_]+'
    text = re.sub(utl_regex_3,'urlplaceholder', text)
    
   
    
    
    # Contractions
    text = re.sub(r"what's", 'what is ', text)
    text = re.sub(r"can't", 'cannot', text)
    text = re.sub(r"\'s",' ', text)
    text = re.sub(r"\'ve", ' have ', text)
    text = re.sub(r"n't", ' not ', text)
    text = re.sub(r"im", 'i am ', text)
    text = re.sub(r"i'm", 'i am ', text)
    text = re.sub(r"\'re", ' are ', text)
    text = re.sub(r"\'d", ' would ', text)
    text = re.sub(r"\'ll", ' will ', text)
                  
    # Operations and special words  
    text = re.sub('#',' ', text)         
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub('foof', 'food', text)
    text = re.sub('msg', 'message', text)
    text = re.sub(' u ', 'you', text)
    
    # Ponctuation Removal
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    text = text.split()
    stop_words = stopwords.words("english")
    text = [tok for tok in text if tok not in stop_words]
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(w) for w in text]
    return ' '.join(text)

In [46]:
# Cleaning Text
df['Review'] = df['Review'].map(lambda x: clean_text(x))

In [73]:
vocabulary_size = 7000
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(df['Review'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['Review'])


In [74]:
MAXLEN = 50
X = pad_sequences(sequences, maxlen=MAXLEN)
type(X)
X.shape[1]

(6136, 50)

In [75]:
y = np.array(df[[ 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability','Polarity']])



In [82]:
def get_model(n_inputs, n_outputs):

  #batch_size = 256 # It is the sample size of inputs to be processed at each training stage. 
  embedding_vecor_length = 32
  model = Sequential()
  model.add(Embedding(vocabulary_size, embedding_vecor_length, input_length=n_inputs))
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(LSTM(100))
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model

In [83]:
def evaluate_model(X,y):
  results_test = []
  results_train =[]
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta = 0.01)
  n_inputs, n_outputs = X.shape[1], y.shape[1]
  cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
  for train_ix,test_ix in cv.split(X):
    X_train, X_test = X[train_ix], X[test_ix]
    y_train,y_test = y[train_ix],y[test_ix]
    
    model = get_model(n_inputs, n_outputs)
    
    model.fit(X_train,y_train,verbose = 0,epochs = 10,callbacks = callback)
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)

    train_log_loss = log_loss(y_train, yhat_train)
    test_log_loss = log_loss(y_test,yhat_test)
    
    results_train.append(train_log_loss)
    results_test.append(test_log_loss)

  return results_train,results_test,model

In [84]:
results_train,results_test,model = evaluate_model(X, y)
print(results_train)
print(results_test)

[2.751100483516682, 2.7974357400571384, 2.910864032014224, 2.853275274075105, 2.8703375012653862, 2.864203570281575, 2.770376074618272, 2.8946263545592514, 2.7933870707869066, 2.9165784356659166, 2.8479226268554667, 2.8180914672444857, 2.836650442071737, 2.8317647254480365, 2.952050908389172]
[3.8546906593301205, 4.145234652173257, 4.040531698192342, 3.9817493989784345, 3.9749171501655547, 3.995160577691339, 4.091125427656477, 3.8808331556235665, 4.085279743168154, 4.0464596153398595, 3.9803564865081436, 3.9341608039372904, 3.998474584814972, 3.880317104827027, 4.122433203211041]


In [97]:
test_data = pd.read_csv('/content/test.csv')
test_data.columns

Index(['Id', 'Review', 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity'],
      dtype='object')

In [98]:
test_data['Review'] = test_data['Review'].map(lambda x: clean_text(x))

In [99]:
vocabulary_size = 7000
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(test_data['Review'])
word_index = tokenizer.word_index
sequences_test = tokenizer.texts_to_sequences(test_data['Review'])

In [101]:
MAXLEN = 50
X_test = pad_sequences(sequences_test, maxlen=MAXLEN)
X_test.shape

(2631, 50)

In [102]:
test_prediction = model.predict(X_test)

In [104]:
pred_data = pd.DataFrame(test_prediction, columns = ['Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity'])

pred_data.shape

(2631, 12)

In [105]:
pred_data.head(5)

,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0.010212,0.010287,0.175081,0.252680,0.065210,0.003805,0.038869,0.024354,0.000767,0.310542,0.285067,0.988957
1,0.513343,0.337072,0.019269,0.037431,0.019709,0.078994,0.049075,0.033312,0.022831,0.006639,0.004605,0.004882
2,0.004256,0.001874,0.365114,0.202702,0.059696,0.002006,0.009375,0.043354,0.000075,0.183971,0.963210,0.989504
3,0.016257,0.003547,0.095743,0.020826,0.089481,0.971169,0.313290,0.008690,0.000810,0.019101,0.020001,0.984460
4,0.006093,0.006656,0.017355,0.059409,0.029971,0.866111,0.001200,0.003754,0.001742,0.006696,0.032407,0.994306


In [106]:
pred_data.to_csv('Submission1.csv', index=False)